In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
TOPN_candidate = 100
SUBSETNUM = 2
SETS = [2, 3]
predTypes = ['clicks', 'carts', 'orders']
candNote = 'covisit_20_20_20_newSuggester2_Covwgt_timeWgt'
featureNote = 'norm'
data4XgbNote = 'covisit_20_20_20_newSuggester2_Covwgt_timeWgt'
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [4]:
for SET in SETS:
    candidatesSavePath = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{candNote}/'
    data4xgb_path = outputPath + f'data4xgb_set{SET}_top_{TOPN_candidate}_{data4XgbNote}/'
    ! mkdir {data4xgb_path}
    featuresPath = outputPath + f'features_set{SET}_{featureNote}/'
    item_features = pd.read_parquet(featuresPath + 'item_features.pqt')
    user_features = pd.read_parquet(featuresPath + 'user_features.pqt')
    userItem_features = pd.read_parquet(featuresPath + 'userItem_features.pqt')

    
    for predType in predTypes:
        print('type:', predType)
        for s in range(SUBSETNUM):
            print('sub:', s)
            sub = pd.read_parquet(candidatesSavePath + f'{predType}Candidate_{s}.pqt')
            sub['session'] = sub.session.astype('int32')
            sub['order_by_rule'] = sub.groupby('session').cumcount(ascending=False)
            sub = sub.merge(item_features, left_on='aid', right_index=True, how='left').fillna(-1)
            sub = sub.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)
            sub = sub.merge(userItem_features, left_on=['session', 'aid'], right_index=True, how='left').fillna(0)
            if SET == 1 or SET ==2:
                if SET == 1:
                    val_labels_pd = pd.read_parquet('../input/split_2-1_pqt/test_labels.parquet')
                else:
                    val_labels_pd = pd.read_parquet('../input/splited/test_labels.parquet')
                val_labels_type = val_labels_pd.loc[val_labels_pd['type'] == predType]
                aids = val_labels_type.ground_truth.explode().astype('int32').rename('aid')
                val_labels_type = val_labels_type[['session']].astype('int32')
                val_labels_type = val_labels_type.merge(aids, left_index=True, right_index=True, how='left')
                val_labels_type[predType[:-1]] = 1
                sub = sub.merge(val_labels_type, on=['session','aid'],how='left').fillna(0)
            sub.to_parquet(data4xgb_path + f'data4xgb_{predType}_{s}.pqt')


type: clicks
sub: 0
sub: 1
type: carts
sub: 0
sub: 1
type: orders
sub: 0
sub: 1
type: clicks
sub: 0
sub: 1
type: carts
sub: 0
sub: 1
type: orders
sub: 0
sub: 1
